In [ ]:
# Import libraries
# DL libraries
import torch

# User libraries
from util import config, model_management
from models.unet_model import UNet

In [ ]:
# Get the model
model = UNet(n_channels = 3, n_classes = 1)
model.to(config.DEVICE)

# Get and set the saved model parameters
model_parameters = model_management.get_artifact_model_weights()
model.load_state_dict(model_management)

# Place model in eval mode
model.eval()